# Homework 1

In [ ]:
import requests
import feedparser
import tqdm
import time
import pymongo
import json
from pymongo import MongoClient

client = pymongo.MongoClient('localhost',27017)
mydb = client["myhomework"] 
collection = mydb["corrigé"]

# Read JSON file
with open(r"C:\Users\Samira\Downloads\pubmed_cleaned.json", "r") as file:
    data = json.load(file)

# Insert documents into MongoDB collection
collection.insert_many(data)

##1) Create an index, explain your choice of key.
collection.create_index([ ("id",1) ])

##2) Delete every paper that was published prior 2019
query={"date":{'$regex':'^year 201[^9]','$options':'i'}}
mydb.corrigé.delete_many(query)

##3) How many paper have a single author ? 
# With number of arrays
N_single_auth = collection.count_documents( { "author": { '$size':1 } } );
#print(N_single_auth)
#Two author
N_Two_Auth = collection.count_documents( { "author": { '$size':2} } );
#print(N_Two_Auth)
# With Regex method
N_single_Auth_regex = collection.count_documents({ "authors": {'$regex':"^(?:(?!name ml).)*name ml(?!.*name ml).*$" }});
#print(N_single_Auth_regex)

##4) What's the last paper inserted in the db ?
Last_one = collection.find().sort("_id", -1).limit(1)
#print(list(Last_one))

##5) Find the number of articles with "technology" in their summary or title.
query = { "meshwords": { "$exists": False } }
docs = collection.count_documents(query)
#print("Number of articles with null meshwords:", docs)

## 7) What's the number of articles that have atleast one affiliation AND meshwords.
query = { "$and": [ { "affiliation": { "$exists": True } }, { "meshwords": { "$exists": True } } ] }
docs = collection.count_documents(query)
#print("Number of articles:", docs)

##8) How many articles have a publishing date after 2020 ?.
Nomber_article = collection.count_documents({'published' :{'$regex': ".*2020.*"}})
#print("Numbers of articles : ", Nomber_article)
   
##10) Check for any duplicates.
len("field")!= len(set ("field"))

##11) Remove every articles where summary starts with an "R".
collection.delete_many({'summary': {'$regex': "\"R.*"}})

##13) Create 500 random samples of the dataset, compute a statistics that you are interested in and check how it behaves through the different samples
import random
import numpy as np
articles=[{"array":list(np.random.random(500))},
          {"array":list(np.random.random(500))},
          {"array":list(np.random.random(500))}]
collection.insert_many(articles)
docs=collection.find()

